Fusion App template has wrong SAML parameters due to which SSO does not work. Follow the steps below to patch an already configured app to have correct SSO parameters.

**Note**: The steps apply to an app and doesn't fix the template.

## Pre-requisties -

Create a trusted app in IDCS with **Identity Domain Administrator** privilege. Gather it's `client ID` and `secret`. This is needed to call the *patch* operation.

In [ ]:
import requests, json, base64, urllib.parse

In [ ]:
tenant = ""
clientId = ''
clientSecret = ''

domain = "identity.oraclecloud.com"
port = "443"
baseurl = 'https://' + tenant + '.' + domain + ':' + port
configurl =  baseurl + '/.well-known/idcs-configuration'

Default App name is provided. If you have changed the App name while adding from template, change it below.

In [ ]:
targetappname = "Oracle Fusion Applications Release 13"

* Generate the Access Token 
* Search the App

In [ ]:
config = None
response = requests.get(configurl).text
config = json.loads(response)
tokenurl = config["openid-configuration"]["token_endpoint"]

grant_type = "client_credentials"
scope = "urn:opc:idm:__myscopes__"

basicauthHeader = base64.b64encode(bytes(clientId + ":" + clientSecret, 'utf-8')) 
reqdata = {'grant_type': grant_type, 'scope' : scope}
reqheaders = {'Authorization': 'Basic ' + basicauthHeader.decode("utf-8"), 'content-type': 'application/x-www-form-urlencoded'}

response = requests.post(tokenurl, data = reqdata, headers = reqheaders)
accesstoken = response.json()["access_token"]
print(accesstoken)

In [ ]:
appsurl = baseurl + "/admin/v1/Apps?attributes=urn:ietf:params:scim:schemas:oracle:idcs:extension:samlServiceProvider:App,aliasApps&filter=displayName eq \"" + targetappname + "\""
targetappid = None

appsheaders = {'Authorization': 'Bearer ' + accesstoken, 'content-type': 'application/json'}
appslist = requests.get(appsurl, headers = appsheaders).json()["Resources"]
fapp = appslist[0]

print(fapp["id"])

In [ ]:
print(fapp['urn:ietf:params:scim:schemas:oracle:idcs:extension:samlServiceProvider:App'])
applinks = fapp['aliasApps']
applinkids = [i['value'] for i in applinks]

In [ ]:
apppatchurl = baseurl + "/admin/v1/Apps/" + fapp["id"]
patchpayload = { "schemas": [
    "urn:ietf:params:scim:api:messages:2.0:PatchOp"
  ],
  "Operations": [
{
      "op": "replace",
      "path": "urn:ietf:params:scim:schemas:oracle:idcs:extension:samlServiceProvider:App:assertionConsumerUrl",
      "value":  "https://${app.serviceParams[name='tenant']}-login.${app.serviceParams[name='domain']}/oam/server/fed/sp/sso"
      
    },
    {
      "op": "replace",
      "path": "urn:ietf:params:scim:schemas:oracle:idcs:extension:samlServiceProvider:App:logoutRequestUrl",
      "value":  "https://${app.serviceParams[name='tenant']}-login.${app.serviceParams[name='domain']}/oamfed/sp/samlv20"
      
    },
    {
      "op": "replace",
      "path": "urn:ietf:params:scim:schemas:oracle:idcs:extension:samlServiceProvider:App:logoutResponseUrl",
      "value":  "https://${app.serviceParams[name='tenant']}-login.${app.serviceParams[name='domain']}/oamfed/sp/samlv20"
      
    },
    {
      "op": "replace",
      "path": "urn:ietf:params:scim:schemas:oracle:idcs:extension:samlServiceProvider:App:partnerProviderId",
      "value":  "https://${app.serviceParams[name='tenant']}-login.${app.serviceParams[name='domain']}:443/oam/fed"
      
    }
]}

response = requests.patch(apppatchurl, data = json.dumps(patchpayload), headers = appsheaders)

print(response)

In [ ]:
for applink in applinkids:
    t = requests.get(baseurl + "/admin/v1/Apps/" + applink, headers = appsheaders).json()

    newpageurl = {
      targetappname + ' CRM': lambda: "https://${app.serviceParams[name='tenant']}-fa-ext.${app.serviceParams[name='domain']}/customer/faces/CrmFusionHome",
      targetappname + ' SCM': lambda: "https://${app.serviceParams[name='tenant']}-fa-ext.${app.serviceParams[name='domain']}/costManagement/faces/FuseWelcome",
      targetappname + ' HCM': lambda: "https://${app.serviceParams[name='tenant']}-fa-ext.${app.serviceParams[name='domain']}/hcmCore/faces/HcmFusionHome",
      targetappname + ' ERP': lambda: "https://${app.serviceParams[name='tenant']}-fa-ext.${app.serviceParams[name='domain']}/ledger/faces/FuseWelcome"
    }[t["displayName"]]()
    
    print(newpageurl)
    
    linkpatchpayload = { "schemas": ["urn:ietf:params:scim:api:messages:2.0:PatchOp"],
                          "Operations": [
                                {
                                    "op": "replace",
                                    "path": 'landingPageUrl',
                                    "value":  newpageurl
                                }
                        ]}
    
    res = requests.patch(baseurl + "/admin/v1/Apps/" + applink, data = json.dumps(linkpatchpayload), headers = appsheaders)
    print(res.text)
    
    
    